# Stochastic simulation assignment 2
## Peter Voerman and Nick van Santen
### 11749547 and ...

In [23]:
import math
import numpy as np
import random
import simpy
import bisect

In [45]:

class Server():

    def __init__(self, env, n_machines):
        self.env = env
        self.machine = simpy.PriorityResource(env, n_machines)

    def run_task(self, waiting_time):
        yield self.env.timeout(waiting_time)

    @property
    def queue_length(self):
        return len(self.machine.queue)


def process_task(env, server, data, waiting_times, duration):
    waiting_time = generate_random_markov(duration)
    #waiting_times.append(waiting_time)
    bisect.insort(waiting_times, waiting_time)
    waiting_time = waiting_times[0]
    waiting_times.pop(0)

    #print(f"{task} added to server at {env.now}. Queue length: {server.queue_length}")
    time_at_queue = env.now
    with server.machine.request(priority=1) as request:
        yield request

        #print(f"{task} is being processed at {env.now}")
        time_start_process = env.now
        yield env.process(server.run_task(waiting_time))
        #print(f"{task} is completed at {env.now}")
        time_end_process = env.now
        

    data["wait_times"].append(time_start_process - time_at_queue)
    data["process_times"].append(time_end_process - time_start_process)

def setup(env, init, data):
    INIT_TASKS = init[0]
    N_MACHINES = init[1]

    MARKOV_TASK_DURATION = init[2]
    MARKOV_TASK_ARRIVAL = init[3]

    server = Server(env, N_MACHINES)
    waiting_times = []

    for i in range(INIT_TASKS):
        env.process(process_task(env, server, data, waiting_times, MARKOV_TASK_DURATION))

    while True:
        yield env.timeout(generate_random_markov(MARKOV_TASK_ARRIVAL))
        i += 1
        # if server.queue_length > 0:
        #     print(server.machine.queue)
        env.process(process_task(env, server, data, waiting_times, MARKOV_TASK_DURATION))


def generate_random_markov(lamda):
    """
    Markov CDF: y = 1 - e^(-lamda t)
    The y value has a range of 0 to 1, which we can sample.
    Thus we can obtain a random t value by sampling y

    t = - ln(1 - y) / lamda
    """

    r = random.random()
    return -math.log(1 - r) / lamda


In [36]:
def run(init):
    data = {
    "wait_times": [],
    "process_times": [],
    }
    env = simpy.Environment()
    env.process(setup(env, init, data))
    env.run(until=MAX_SIM_TIME)
    
    
    print(f"Avg wait time: {np.mean(data['wait_times'])}")
    print(f"Avg process time: {np.mean(data['process_times'])}")

In [48]:
INIT_TASKS = 4
N_MACHINES = 1
MAX_SIM_TIME = 100000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 0.9



init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL]

run(init)

Avg wait time: 9.299990229046989
Avg process time: 1.0019601564187075


In [38]:
INIT_TASKS = 4
N_MACHINES = 2
MAX_SIM_TIME = 10000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 1.8


init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL]

run(init)

Avg wait time: 642.2247043177842
Avg process time: 1.2831863549881068


In [27]:
INIT_TASKS = 4
N_MACHINES = 2
MAX_SIM_TIME = 10

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 1.8


init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL]

run(init)

[1.749585746570078]
[0.8966098907180297, 1.749585746570078]
[0.16953797145259827, 0.8966098907180297, 1.749585746570078]
[0.16953797145259827, 0.4005365349804915, 0.8966098907180297, 1.749585746570078]
[0.08234912290023096, 0.8966098907180297, 1.749585746570078]
[0.08234912290023096, 0.8966098907180297, 1.405905908621783, 1.749585746570078]
[1.1635788951927017, 1.405905908621783, 1.749585746570078]
[0.2376282250722193, 1.1635788951927017, 1.405905908621783, 1.749585746570078]
[0.2376282250722193, 1.1635788951927017, 1.405905908621783, 1.749585746570078, 3.191326322082998]
[0.2376282250722193, 1.1635788951927017, 1.405905908621783, 1.6874911968942081, 1.749585746570078, 3.191326322082998]
[0.17342062525796054, 0.2376282250722193, 1.1635788951927017, 1.405905908621783, 1.6874911968942081, 1.749585746570078, 3.191326322082998]
[1.1635788951927017, 1.405905908621783, 1.4895548904996352, 1.6874911968942081, 1.749585746570078, 3.191326322082998]
[0.6205162143384024, 1.4895548904996352, 1.687